In [ ]:
from time import time, sleep
from json import loads
from os.path import exists
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from pickle import dump,load
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

In [ ]:
class Concert(object):
    def __init__(self, 
                 driver_path, damai_url, target_url, session, price, ticket_num, viewer_person):
        self.status = 0                      # 流程状态标记                      
        self.num = 0                         # 尝试次数
        self.time_start = 0                  # 开始时间
        self.time_end = 0                    # 结束时间
        self.driver_path = driver_path       # 浏览器驱动地址
        self.damai_url = damai_url           # 大麦网官网网址
        self.target_url = target_url         # 目标购票网址（具体到城市）
        self.session = session               # 场次优先级序号
        self.price = price                   # 票档优先级序号
        self.ticket_num = ticket_num         # 购买票数
        self.viewer_person = viewer_person   # 观影人优先级序号
        self.driver = None                   # 重置

    # 获取账号的cookie信息
    def get_cookie(self):
        self.driver.get(self.damai_url)
        print('---正在点击登录---')
        self.driver.find_element(by = By.CLASS_NAME, value = 'login-user').click()
        while self.driver.title.find('大麦网-全球演出赛事官方购票平台') != -1:
            sleep(1)
        print('###请手动登录您的大麦账号###')
        while self.driver.title == '大麦登录':
            sleep(1)
        dump(self.driver.get_cookies(), open('cookies.pkl', 'wb'))
        print('---Cookie获取成功---')
        self.driver.quit()

    # 注入获取的cookie信息
    def set_cookie(self):
        try:
            cookies = load(open('cookies.pkl', 'rb'))
            for cookie in cookies:
                cookie_dict = {
                    'domain': '.damai.cn',   # 保证所有子域名间共享cookie,避免后续cookie无效
                    'name': cookie.get('name'),
                    'value': cookie.get('value'),
                    "expires": "",
                    'path': '/',
                    'httpOnly': False,
                    'HostOnly': False,
                    'Secure': False}
                self.driver.add_cookie(cookie_dict)
            print('---Cookie载入成功---')
        except Exception as e:
            print(f'***Error: 注入cookie发生错误***:{e}')

    # 打开商品详情页
    def login(self):
        print('---进入商品详情网页---')
        self.driver.get(self.target_url)
        WebDriverWait(self.driver, 10, 0.1).until(EC.title_contains('商品详情'))
        print('---正在登录---')
        self.set_cookie()

    def enter_concert(self):
        self.time_start = time()   # 记录开始时间
        if not exists('cookies.pkl'):   # 判断cookie是否已经获取
            service = Service(self.driver_path)
            self.driver = webdriver.Chrome(service = service)
            self.get_cookie()   # 调用获取cookie函数
            print('---正在重启浏览器---')
        
        # 配置浏览器
        options = webdriver.ChromeOptions()
        prefs = {"profile.managed_default_content_settings.images": 2,   # 禁用图片
                 "profile.managed_default_content_settings.javascript": 1,   # 启用 JavaScript
                 'permissions.default.stylesheet': 2}   # 禁用 CSS
        options.add_experimental_option('prefs', prefs)   # 禁用资源加载
        mobile_emulation = {"deviceName": "Nexus 6"}
        options.add_experimental_option("mobileEmulation", mobile_emulation)   # 模拟移动端
        # 隐藏自动化
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        options.add_argument('--log-level=3')   # 日志级别,仅显示错误日志
        options.set_capability("pageLoadStrategy", "eager")   # 更换等待策略为无需等待所有资源

        service = Service(self.driver_path)
        self.driver = webdriver.Chrome(service = service, options = options)

        self.login()
        self.driver.refresh()   # 刷新页面,保证账号为登录状态

    # 判断函数
    def isClassPresent(self, item, name, ret = False):
        try:
            result = item.find_element(by = By.CLASS_NAME, value = name)
            if ret:
                return result
            else:
                return True
        except:
            return False

    # 实现场次/票档/购票数等的选择
    def choose_ticket(self):
        print('---进入抢票页面---')
        
        while self.driver.title.find('确认购买') == -1:
            self.num += 1   # 尝试次数
            if self.driver.current_url.find('mclient.alipay.com') != -1:
                break
            
            # 页面初始化检查确认加载成功
            try:
                WebDriverWait(self.driver, 10, 0.1).until(
                    lambda driver: driver.execute_script('return document.readyState') == 'complete')
            except:
                raise Exception("***Error: 页面加载超时***")
            try:
                box = WebDriverWait(self.driver, 5, 0.1).until(
                    EC.presence_of_element_located((By.ID, 'root')))
            except:
                raise Exception("***Error: 页面中ID为root的整体布局元素不存在或加载超时***")
            
            # 检查并处理温馨提示
            try:
                health_info = WebDriverWait(self.driver, 1, 0.1).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'health-info-container')))
                print('---检测到温馨提示---')
                health_info_box = health_info.find_element(by = By.CLASS_NAME, value = 'scroll-view-health')
                print('---正在模拟向上滑动阅读温馨提示内容---')
                self.driver.execute_script(
                    'arguments[0].scrollTop = arguments[0].scrollHeight', health_info_box)   # 模拟向上滑动
                sleep(0.5)
                print('---模拟滑动成功---')
                print('---正在点击“知道了”按钮---')
                know_button = health_info.find_element(by = By.CLASS_NAME, value = 'health-info-button')
                know_button.click()
                print('---成功点击“知道了”按钮---')
            except TimeoutException:
                print('---不存在温馨提示---')
            except NoSuchElementException as e:
                print(f'***Error: 温馨提示处理异常:{e}***')
            
            # 检查并处理实名制提示
            try:
                realname_info = WebDriverWait(self.driver, 1, 0.1).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'bui-show-real-name-modal-content')))
                print('---检测到实名制观演提示---')
                print('---正在点击“知道了”按钮---')
                know_button = realname_info.find_element(
                    by = By.CLASS_NAME, value = 'bui-show-real-name-modal-content-btn-close')
                know_button.click()
                print('---成功点击“知道了”按钮---')
            except TimeoutException:
                print('---不存在实名制观演提示---')
            except NoSuchElementException as e:
                print(f'***Error: 实名制观演提示处理异常:{e}***')

            try:
                buy_button = box.find_element(by = By.CLASS_NAME, value = 'buy-button')
                buy_button_text = buy_button.text
                print('---定位购买按钮成功---')
            except Exception as e:
                print(f'***Error: 定位购买按钮失败***:{e}')

            if ('预约抢票' in buy_button_text) or ('即将开抢' in buy_button_text):
                raise Exception("---尚未开售，刷新等待---")
            if '缺货' in buy_button_text:
                raise Exception("---已经缺货，刷新等待---")
            if ('取消' in buy_button_text) or ('渠道不支持' in buy_button_text):
                raise Exception("---取消或者需要在APP端操作---")
            
            sleep(0.1)
            buy_button.click()
            print('---点击购买按钮---')
            box = WebDriverWait(self.driver, 1, 0.1).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.sku-pop-wrapper')))

            # 选择场次和票档
            try:
                # 选择场次
                session = WebDriverWait(self.driver, 1, 0.1).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'bui-dm-sku-card')))
                session_list = session.find_elements(by = By.CLASS_NAME, value = 'bui-dm-sku-card-item')
                toBeClicks = []
                for i in self.session:
                    if i > len(session_list):
                        i = len(session_list)
                    j = session_list[i-1]
                    k = self.isClassPresent(j, 'item-tag', True)
                    if k:
                        if k.text == '无票':
                            continue
                        elif k.text == '预售':
                            toBeClicks.append(j)
                            break
                    else:
                        toBeClicks.append(j)
                        break

                # 点击场次,多场次需要先点击才会出现票档 单场次或者已点击场次再次点击不会取消场次选择
                for i in toBeClicks:
                    i.click()
                    print('---场次选定---')
                    sleep(0.05)

                # 选择票档
                price = WebDriverWait(self.driver, 1, 0.1).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'sku-tickets-card')))
                price_list = price.find_elements(by = By.CLASS_NAME, value = 'bui-dm-sku-card-item')
                toBeClicks = []
                for i in self.price:
                    if i > len(price_list):
                        i = len(price_list)
                    j = price_list[i-1]
                    k = self.isClassPresent(j, 'item-tag', ret = True)
                    if k:
                        continue   # 只存在缺货登记的情况 若k存在则跳过该票档
                    else:
                        toBeClicks.append(j)
                        break

                for i in toBeClicks:
                    i.click()
                    print('---票档选定---')
                    sleep(0.1)
                
                buy_button = box.find_element(by = By.CLASS_NAME, value = 'sku-footer-buy-button')
                buy_button_text = buy_button.text
                if buy_button_text == '':
                    raise Exception("***Error: 提交票档按钮文字获取为空,适当调整 sleep 时间***")
                
                try:
                    WebDriverWait(self.driver, 1, 0.1).until(
                        EC.presence_of_element_located((By.CLASS_NAME, 'bui-dm-sku-counter')))
                except:
                    raise Exception("***Error: 购票按钮未开始***")
            except Exception as e:
                raise Exception(f"***Error: 选择场次or票档不成功***:{e}")

            # 选择购票数与提交购票
            try:
                ticket_num_up = box.find_element(by = By.CLASS_NAME, value = 'plus-enable')
            except:
                if '选座' in buy_button_text:   # 选座购买没有票数加减按钮
                    buy_button.click()
                    self.status = 1
                    print('###请自行选择位置和票价###')
                    break
                elif buy_button_text == '提交缺货登记':
                    raise Exception('###票已被抢完，持续捡漏中...或请关闭程序并手动提交缺货登记###')
                else:
                    raise Exception('***Error: ticket_num_up 位置找不到***')

            if buy_button_text == '立即预订' or buy_button_text == '立即购买' or buy_button_text == '确定':
                for i in range(self.ticket_num - 1):
                    ticket_num_up.click()   # 增加票数
                buy_button.click()   # 点击购票
                self.status = 2
                WebDriverWait(self.driver, 2, 0.1).until(EC.title_contains('确认购买'))
                break
            else:
                raise Exception(f'***Error: 未定义按钮:{buybutton_text}')
    
    # 选择观影人并提交订单
    def check_order(self):
        # 选择观影人
        if self.status in [1, 2]:
            WebDriverWait(self.driver, 5, 0.1).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="dmViewerBlock_DmViewerBlock"]/div[2]/div/div')))
            people = self.driver.find_elements(
                by = By.XPATH, value = '//*[@id="dmViewerBlock_DmViewerBlock"]/div[2]/div/div')
            sleep(0.2)
            
            for i in self.viewer_person:
                if i > len(people):
                    break
                j = people[i-1]
                j.click()
                print('---观影人选定---')
                sleep(0.5)

            # 提交订单尝试
            WebDriverWait(self.driver, 5, 0.1).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="dmOrderSubmitBlock_DmOrderSubmitBlock"]/div[2]/div/div[2]/div[2]/div[2]')))
            confirmBtn = self.driver.find_element(
                By.XPATH,'//*[@id="dmOrderSubmitBlock_DmOrderSubmitBlock"]/div[2]/div/div[2]/div[2]/div[2]')
            sleep(0.5)
            confirmBtn.click()
            print('---正在尝试提交订单---')

            while True:
                try:
                    WebDriverWait(self.driver, 5, 0.1).until(EC.title_contains('支付宝'))
                    print('---订单提交成功---')
                    self.status = 3
                    self.time_end = time()
                    break
                except:    # 需要人工判断
                    step = input('\n###\n1.成功跳转到支付宝付款页面\n2.未知，没跳转到支付宝界面，尝试重新抢票\n3.未知，退出脚本\n###\n请输入当前状态：')
                    if step == '1':
                        print('---订单提交成功---')
                        self.status = 3
                        self.time_end = time()
                        break
                    elif step == '2':
                        print('---尝试重新抢票---')
                        return True
                    elif step == '3':
                        print('---脚本退出成功---')
                        return False
                    else:
                        raise Exception('***Error: 未知输入***')

In [ ]:
if __name__ == '__main__':
    try:
        with open('config.json', 'r', encoding = 'utf-8') as f:
            config = loads(f.read())
        con = Concert(config['driver_path'], config['damai_url'], config['target_url'], config['session'], config['price'], config['ticket_num'], config['viewer_person'])
        con.enter_concert()   # 进入到抢购商品详情页面
    except Exception as e:
        print(f'***Error: 准备或进入阶段发生错误***:{e}')
        exit(1)

    while True:
        try:
            con.choose_ticket()
            print('---正在进入观影人选择界面---')
            retry = con.check_order()
            if not retry:
                break
        except Exception as e:
            con.driver.get(con.target_url)
            print(e)
            continue

    if con.status == 3:
        print(f'---通过{con.num}次尝试,耗时{con.time_end - con.time_start:.2f}秒,成功为您抢票!请及时确认订单信息并完成支付!---')